# Imports & Data

In [1]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix, diags, identity
from scipy.spatial.distance import cdist

from pathlib import Path
from tqdm import tqdm

import scipy.sparse as sp
from scipy.sparse.linalg import svds, spsolve_triangular, eigsh, splu


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler, OneHotEncoder


from dataprep import (
    generate_interactions_matrix,
    leave_last_out,
    transform_indices,
    verify_time_split,
    reindex_data,
)
from evaluation import (
    topn_recommendations,
    model_evaluate,
    downvote_seen_items,
    calculate_rmse,
    postprocess_scores,
)

from catboost import CatBoostRegressor, Pool, EFstrType
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.simplefilter("ignore", category=ConvergenceWarning)

In [2]:
np.random.seed(42)

# Helpers

In [3]:
gower_w_pass = {"item_feats": None, "user_feats": None}

In [4]:
def split_by_time(data: pd.DataFrame, split_time, userid="userid", timeid="timestamp"):

    test_users = data[data[timeid] > split_time][userid].unique()

    return data[~data[userid].isin(test_users)], data[data[userid].isin(test_users)]

In [5]:
def get_gower_sim(A, w=None):

    mins = A.min(axis=0)
    maxes = A.max(axis=0)
    ranges = maxes - mins
    ranges[ranges == 0] = 1.0

    An = (A - mins) / ranges

    _, d = An.shape
    w = w if w is not None else np.ones(d)
    assert w.size == d

    diffs = An[:, None, :] - An[None, :, :]
    diffs = np.abs(diffs)
    diffs *= w

    similarity = 1 - np.sum(diffs, axis=2) / np.sum(w)

    similarity = csr_matrix(similarity)
    similarity.setdiag(0)
    similarity.eliminate_zeros()

    return similarity

# Models

In [6]:
def cosine_similarity_zd(A):
    """Build cosine similarity matrix with zero diagonal."""
    A = csr_matrix(A)
    similarity = cosine_similarity(A, dense_output=False)
    similarity.setdiag(0)
    similarity.eliminate_zeros()
    return similarity


def get_gower_sim(A, w=None):
    mins = A.min(axis=0)
    maxes = A.max(axis=0)
    ranges = maxes - mins
    ranges[ranges == 0] = 1.0

    A = (A - mins) / ranges

    n, d = A.shape
    w = w if w is not None else np.ones(d)
    assert w.size == d

    Aw = A * w
    D = cdist(Aw, Aw, metric="cityblock") / np.sum(w)

    similarity = 1 - D
    similarity = csr_matrix(similarity)
    similarity.setdiag(0)
    similarity.eliminate_zeros()
    return similarity


def sparse_dropout(A, p=0.5):
    # Randomly zero p of all elements
    A_coo = A.tocoo()
    nnz = A_coo.nnz
    keep_indices = np.random.choice(nnz, size=int(nnz * (1 - p)), replace=False)
    return csr_matrix(
        (A_coo.data[keep_indices], (A_coo.row[keep_indices], A_coo.col[keep_indices])),
        shape=A.shape,
    )


def topk(A, p=0.5):
    # Leave only p of all elements per row
    A = A.copy()
    for i in range(A.shape[0]):
        start, stop = A.indptr[i], A.indptr[i + 1]
        row = A.data[start:stop]
        k = int(len(row) * p)
        keep_mask = np.zeros(len(row), dtype=bool)
        keep_mask[np.argpartition(row, -k)[-k:]] = True
        A.data[start:stop][~keep_mask] = 0
    A.eliminate_zeros()
    return A


def build_iknn_model(config, data, data_description):
    item_similarity = None
    if config.get("gower", False):
        feats = data_description["item_features_mtx"]
        item_similarity = get_gower_sim(
            feats, w=config.get("gower_w", gower_w_pass)["item_feats"]
        )
    else:
        item_similarity = cosine_similarity_zd(data_description["item_features_mtx"])
    if config.get("sampling", True):
        # we leave half of the samples and then only highest rated half
        item_similarity = sparse_dropout(item_similarity, config.get("p_dropout", 0.5))
        item_similarity = topk(item_similarity, config.get("p_topk", 0.5))
    return item_similarity


@postprocess_scores
def iknn_model_scoring(params, testset, testset_description):
    item_similarity = params
    test_mtx = generate_interactions_matrix(
        testset, testset_description, rebase_users=True
    )
    scores = test_mtx @ item_similarity
    return scores.toarray()

In [7]:
def random_scoring(testset, testset_description):
    test_mtx = generate_interactions_matrix(
        testset, testset_description, rebase_users=True
    )
    return np.random.rand(*test_mtx.shape)

In [8]:
def easer_build(config, data, data_description):

    gower = config["gower"]
    alpha = config["alpha"]
    l = config["l"]

    A = generate_interactions_matrix(data, data_description)
    I = csr_matrix(np.eye(data_description["n_items"]))

    Y = data_description["item_features_mtx"]
    if gower:
        F = get_gower_sim(Y, w=config.get("gower_w", gower_w_pass)["item_feats"])
    else:
        F = cosine_similarity_zd(Y)
        # F = csr_matrix(Y @ Y.T)
        # F.setdiag(0)

    G = A.T @ A + alpha * F + l * I

    P = np.linalg.inv(G.toarray())
    W = np.eye(A.shape[1]) - P / np.diag(P)
    return W


@postprocess_scores
def easer_scoring(params, testset, testset_description):
    W = params
    A = generate_interactions_matrix(testset, testset_description, rebase_users=True)

    return A @ W

In [9]:
def knn(Y, mode, config, nnbrs=10):
    if mode == "gower":
        S = get_gower_sim(
            Y, w=config.get("gower_w", gower_w_pass)["item_feats"]
        ).toarray()
    else:
        S = cosine_similarity(Y, Y)

    np.fill_diagonal(S, 0.0)

    neighbors = np.argpartition(-S, nnbrs, axis=1)[:, :nnbrs]
    return neighbors


def fsSLIM_build(config, data, data_description):
    nnbrs = config["nnbrs"]
    alpha_reg = config["alpha_reg"]
    beta_reg = config["beta_reg"]
    gower = config["gower"]

    assert alpha_reg + beta_reg > 0

    A = generate_interactions_matrix(data, data_description)
    Y = data_description["item_features_mtx"]

    neighbours = (
        knn(Y, "gower", config, nnbrs) if gower else knn(Y, "cos_sim", config, nnbrs)
    )

    n_items = data_description["n_items"]
    W = np.zeros((n_items, n_items))

    model = ElasticNet(
        alpha=alpha_reg + beta_reg,
        l1_ratio=alpha_reg / (alpha_reg + beta_reg),
        fit_intercept=False,
        positive=True,
        max_iter=50,
        tol=config.get("tol", 1e-2),
    )

    for j in range(n_items):
        idx = neighbours[j]

        X = A[:, idx].toarray()
        y = A[:, j].toarray().ravel()

        model.fit(X, y)

        W[idx, j] = model.coef_

    return W


@postprocess_scores
def fsSLIM_scoring(params, testset, testset_description):
    W = params
    A = generate_interactions_matrix(testset, testset_description, rebase_users=True)
    return A @ W

In [10]:
def make_spd_similarity(sim, gamma=0.9, eps=1e-6):
    n = sim.shape[0]
    I = sp.eye(n, format="csr")

    sim = 0.5 * (sim + sim.T)

    sim = sim.copy()
    sim.setdiag(1.0)

    K = gamma * sim + (1 - gamma) * I

    K = K + eps * I

    return K


def safe_dot_kernel(Y, eps=1e-6):
    Y = Y.astype(np.float64)
    norms = np.linalg.norm(Y, axis=1, keepdims=True)
    norms[norms == 0] = 1.0
    Y = Y / norms
    K = csr_matrix(Y @ Y.T)
    K.setdiag(1.0)
    K = K + eps * sp.eye(K.shape[0], format="csr")
    return K


def sparse_cholesky(A):
    sparse_matrix = A.T @ A
    sparse_matrix += 1e-6 * identity(sparse_matrix.shape[0])
    n = sparse_matrix.shape[0]
    LU = splu(sparse_matrix, diag_pivot_thresh=0.0, permc_spec="NATURAL")

    L = LU.L @ diags(LU.U.diagonal() ** 0.5)

    return L


def hysvd_build(config, data, data_description):
    rank = config.get("rank", 25)
    gamma_s = config.get("gamma_s", 1)
    gamma_k = config.get("gamma_k", 1)
    gower = config.get("gower", False)

    A = generate_interactions_matrix(data, data_description).tocsr()

    Y = data_description["item_features_mtx"]
    X = data_description["user_features_mtx"]
    if gower:
        S_sim = get_gower_sim(Y, w=config.get("gower_w", gower_w_pass)["item_feats"])
        S = make_spd_similarity(S_sim, gamma=gamma_s)
        K_sim = get_gower_sim(X, w=config.get("gower_w", gower_w_pass)["user_feats"])
        K = make_spd_similarity(K_sim, gamma=gamma_k)
    else:
        S_sim = safe_dot_kernel(Y)
        S = make_spd_similarity(S_sim, gamma=gamma_s)
        K_sim = safe_dot_kernel(X)
        K = make_spd_similarity(K_sim, gamma=gamma_k)

    Ls = csr_matrix(np.linalg.cholesky(S.toarray()))
    Lk = csr_matrix(np.linalg.cholesky(K.toarray()))

    # Ls = sparse_cholesky(S)
    # Lk = sparse_cholesky(K)

    M = Lk.T @ A @ Ls

    U_hat, s, VT_hat = svds(M, k=rank)
    idx = np.argsort(-s)
    s = s[idx]
    U_hat = U_hat[:, idx]
    V_hat = VT_hat[idx, :].T

    V = spsolve_triangular(Ls.T.tocsr(), V_hat, lower=False)

    LV = Ls @ V
    RV = spsolve_triangular(Ls.T.tocsr(), V, lower=False)

    return {"LV": LV, "RV": RV, "singular_values": s}


@postprocess_scores
def hysvd_scoring(params, testset, testset_description):
    LV = params["LV"]
    RV = params["RV"]

    A_test = generate_interactions_matrix(
        testset, testset_description, rebase_users=True
    )

    return A_test @ LV @ RV.T

In [11]:
def popularity_scaling(A, alpha):
    item_pop = np.array(A.sum(axis=0)).ravel()
    item_pop[item_pop == 0] = 1.0
    D_alpha = sp.diags(item_pop ** (-alpha))
    return D_alpha


def eigenrec_build(config, data, data_description):
    rank = config.get("rank", 100)
    alpha = config.get("alpha", 0.5)
    gamma = config.get("gamma", 0.8)
    gower = config.get("gower", True)

    A = generate_interactions_matrix(data, data_description).tocsr()

    D_alpha = popularity_scaling(A, alpha)
    A_scaled = A @ D_alpha

    C = A_scaled.T @ A_scaled
    C.setdiag(0)

    Y = data_description["item_features_mtx"]

    if gower:
        S = get_gower_sim(Y, w=config.get("gower_w", gower_w_pass)["item_feats"])
    else:
        S = cosine_similarity_zd(Y)

    W_sim = gamma * S + (1.0 - gamma) * C

    W_sim = W_sim + 1e-6 * sp.eye(W_sim.shape[0], format="csr")

    vals, vecs = eigsh(W_sim, k=rank, which="LA")

    idx = np.argsort(-vals)
    vals = vals[idx]
    vecs = vecs[:, idx]

    return {"Q": vecs, "Lambda": vals}


@postprocess_scores
def eigenrec_scoring(params, testset, testset_description):
    Q = params["Q"]
    Lambda = params["Lambda"]

    A = generate_interactions_matrix(testset, testset_description, rebase_users=True)

    AQ = A @ Q
    scores = (AQ * Lambda) @ Q.T

    return scores

# Dataprep

## MovieLens1M & BX

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer


def encode_features(
    X: pd.DataFrame,
    element_id: str,
    movie_lens_flg: bool = False,
    categorical: list[str] = None,
    numerical: list[str] = None,
    text: list[str] = None,
    max_tfidf_features: int = 50,
    scale_numeric: bool = True,
    limit_categorical: dict = None,
):
    """
    Encode heterogeneous features into a feature DataFrame.

    limit_categorical:
        dict like {"Publisher": 100, "Book-Author": 200}
        keeps top-K categories, others -> 'OTHER'
    """

    parts = []
    if movie_lens_flg and "genres" in X.columns:
        genre_ohe = X["genres"].fillna("").str.get_dummies(sep="|").add_prefix("genre_")
        parts.append(genre_ohe)
    if categorical:
        X_cat_src = X[categorical].copy()

        if limit_categorical:
            for col, top_k in limit_categorical.items():
                if col in X_cat_src.columns:
                    top_values = X_cat_src[col].value_counts().head(top_k).index
                    X_cat_src[col] = X_cat_src[col].where(
                        X_cat_src[col].isin(top_values), "OTHER"
                    )

        ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
        X_cat = ohe.fit_transform(X_cat_src.fillna("NA"))
        cat_cols = ohe.get_feature_names_out(categorical)

        X_cat_df = pd.DataFrame(X_cat, columns=cat_cols, index=X.index)
        parts.append(X_cat_df)
    if numerical:
        X_num_df = X[numerical].copy()

        for col in numerical:
            X_num_df[col] = pd.to_numeric(X_num_df[col], errors="coerce")
            mean_val = X_num_df[col].mean()
            X_num_df[col] = X_num_df[col].fillna(mean_val)

        X_num = X_num_df.to_numpy(dtype=float)

        if scale_numeric:
            scaler = StandardScaler()
            X_num = scaler.fit_transform(X_num)

        X_num_df = pd.DataFrame(X_num, columns=numerical, index=X.index)
        parts.append(X_num_df)
    if text:
        for col in text:
            tfidf = TfidfVectorizer(
                max_features=max_tfidf_features, stop_words="english"
            )
            X_txt = tfidf.fit_transform(X[col].fillna("").astype(str))
            txt_cols = [f"{col}_tfidf_{t}" for t in tfidf.get_feature_names_out()]

            X_txt_df = pd.DataFrame(X_txt.toarray(), columns=txt_cols, index=X.index)
            parts.append(X_txt_df)
    if not parts:
        raise ValueError("No features were encoded")
    features_df = pd.concat(parts, axis=1)
    result = pd.concat(
        [X[[element_id]].reset_index(drop=True), features_df.reset_index(drop=True)],
        axis=1,
    )

    return result

In [13]:
def cold_start_scenario(
    feedback: pd.DataFrame,
    time_quantile: float = 0.8,
    user_col: str = "userid",
    item_col: str = "movieid",
    time_col=None,
):
    if time_col is None:
        time_col = "timestamp"
        feedback[time_col] = np.random.rand(len(feedback))

    training_, testing_h_ = split_by_time(
        feedback, feedback[time_col].quantile(time_quantile), userid=user_col
    )

    training, training_index = transform_indices(
        training_, users=user_col, items=item_col
    )

    _, testset_h_index = transform_indices(testing_h_, users=user_col, items=item_col)

    data_index = {}
    data_index["users"] = pd.Index(
        list(training_index["users"]) + list(testset_h_index["users"]), name=user_col
    )
    data_index["items"] = training_index["items"]

    testset_h = reindex_data(
        data=testing_h_,
        data_index=data_index,
        entities=["items", "users"],
        filter_invalid=True,
    )

    testset, holdout_ = leave_last_out(testset_h, userid=user_col)
    holdout_ = holdout_[holdout_[user_col].isin(testset[user_col])]
    holdout = holdout_.sort_values(user_col)

    return training, data_index, testset, holdout, training_

In [14]:
data_dir = Path("data/")

feedback_ml = pd.read_csv(
    data_dir / "ml-1m/ratings.dat",
    sep="::",
    engine="python",
    header=None,
    names=["userid", "movieid", "rating", "timestamp"],
    encoding="latin-1",
)

items_ml = pd.read_csv(
    data_dir / "ml-1m/movies.dat",
    sep="::",
    engine="python",
    header=None,
    names=["movieid", "title", "genres"],
    encoding="latin-1",
)

users_ml = pd.read_csv(
    data_dir / "ml-1m/users.dat",
    sep="::",
    engine="python",
    header=None,
    names=["userid", "sex", "age", "occupation", "zipcode"],
    encoding="latin-1",
).drop(columns=["zipcode"])

training, data_index, testset, holdout, feedback_val = cold_start_scenario(
    feedback_ml, time_col="timestamp"
)
item_features_ = encode_features(
    items_ml, element_id="movieid", categorical=[], text=["title"], movie_lens_flg=True
)
user_features_ = encode_features(
    users_ml,
    element_id="userid",
    categorical=["sex", "occupation"],
    text=[],
    numerical=["age"],
)

item_features = reindex_data(
    data=item_features_, data_index=data_index, entities=["items"], filter_invalid=True
)
user_features = reindex_data(
    data=user_features_,
    data_index=data_index,
    entities=["users"],
    filter_invalid=True,
)

training_val, data_index_val, testset_val, holdout_val, _ = cold_start_scenario(
    feedback_val, time_quantile=0.5
)
item_features_val = reindex_data(
    data=item_features_,
    data_index=data_index_val,
    entities=["items"],
    filter_invalid=True,
)

/var/folders/vm/nqqr1_9n4yggqqgcpdv46_q40000gn/T/ipykernel_29543/3064727987.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feedback[time_col] = np.random.rand(len(feedback))


In [15]:
data_description = dict(
    users=data_index["users"].name,
    items=data_index["items"].name,
    feedback="rating",
    n_users=len(data_index["users"]),
    n_items=len(data_index["items"]),
    test_users=holdout[data_index["users"].name].values,
    item_features_mtx=item_features.to_numpy(dtype=float),
    user_features_mtx=user_features.to_numpy(dtype=float),
)

data_description_val = dict(
    users=data_index_val["users"].name,
    items=data_index_val["items"].name,
    feedback="rating",
    n_users=len(data_index_val["users"]),
    n_items=len(data_index_val["items"]),
    test_users=holdout_val[data_index_val["users"].name].values,
    item_features_mtx=item_features_val.to_numpy(dtype=float),
    user_features_mtx=None,
)

# Importances

In [16]:
def get_importances(df, user_features, item_features, data_description):
    df = df.copy()
    df = df.merge(item_features, on=data_description["items"], how="left")
    df = df.merge(user_features, on=data_description["users"], how="left")
    cols = list(item_features.columns) + list(user_features.columns)
    feedback_col = data_description["feedback"]
    train_df, test_df = train_test_split(
        df, random_state=42, test_size=0.2, shuffle=True
    )
    train_pool = Pool(train_df[cols], train_df[feedback_col])
    test_pool = Pool(test_df[cols], test_df[feedback_col])
    model = CatBoostRegressor(random_state=42, iterations=1000, verbose=False)
    model.fit(train_pool)
    print(f"MSE: {mean_squared_error(test_pool.get_label(), model.predict(test_pool))}")
    perm = permutation_importance(
        model,
        test_df[cols],
        test_pool.get_label(),
        n_repeats=10,
        random_state=42,
        scoring="neg_mean_squared_error",
    )["importances_mean"]
    perm -= max(0, perm.min())
    shap_vals = model.get_feature_importance(test_pool, type=EFstrType.ShapValues)
    shap = np.abs(shap_vals[:, :-1]).mean(axis=0)
    norm = lambda x: x / sum(x)
    split = lambda x: {
        "item_feats": x[: len(item_features.columns)],
        "user_feats": x[-len(user_features.columns) :],
    }
    return {
        "builtin": split(norm(model.get_feature_importance())),
        "shap": split(norm(shap)),
        "permutation": split(norm(perm)),
    }


importances = get_importances(
    training,
    item_features=item_features,
    user_features=user_features,
    data_description=data_description,
)

MSE: 0.9455215366549271


# Evaluation

In [17]:
def compare_models(
    configs,
    build_fn,
    scoring_fn,
    training,
    testset,
    holdout,
    data_description,
    topn: int = 10,
    model_name: str = None,
):
    res = []

    for config in tqdm(
        configs, desc=f"Running {model_name}" if model_name else "Running ..."
    ):
        params = build_fn(
            config=config, data=training, data_description=data_description
        )

        scores = scoring_fn(params, testset, data_description)

        recs = topn_recommendations(scores, topn=topn)

        metrics = model_evaluate(recs, holdout, data_description, topn=topn)

        row = {}
        if model_name is not None:
            row["model"] = model_name
        else:
            row["model"] = "noname"

        row["is_gower"] = config["gower"]
        row["w_type"] = config.get("w_type", "None")
        row.update(metrics._asdict())

        res.append(row)

    return res

In [18]:
models = {
    "knn": (build_iknn_model, iknn_model_scoring),
    "EASER": (easer_build, easer_scoring),
    "fsSLIM": (fsSLIM_build, fsSLIM_scoring),
    "HySVD": (hysvd_build, hysvd_scoring),
    "EigenRec": (eigenrec_build, eigenrec_scoring),
}

w_types = ["builtin", "shap", "permutation"]

model_configs = {
    "knn": [
        {"sampling": False, "gower": True},
        {"sampling": False, "gower": False},
    ]
    + [
        {
            "sampling": False,
            "gower": True,
            "gower_w": importances[w_type],
            "w_type": w_type,
        }
        for w_type in w_types
    ],
    "EASER": [
        {"alpha": 1, "l": 100, "gower": True},
        {"alpha": 1, "l": 100, "gower": False},
    ]
    + [
        {
            "alpha": 1,
            "l": 100,
            "gower": True,
            "gower_w": importances[w_type],
            "w_type": w_type,
        }
        for w_type in w_types
    ],
    "fsSLIM": [
        {"nnbrs": 50, "alpha_reg": 1e-3, "beta_reg": 1e-1, "gower": True},
        {"nnbrs": 50, "alpha_reg": 1e-3, "beta_reg": 1e-1, "gower": False},
    ]
    + [
        {
            "nnbrs": 50,
            "alpha_reg": 1e-3,
            "beta_reg": 1e-1,
            "gower": True,
            "gower_w": importances[w_type],
            "w_type": w_type,
        }
        for w_type in w_types
    ],
    "HySVD": [
        {"gower": True, "rank": 35},
        {"gower": False, "rank": 35},
    ]
    + [
        {"gower": True, "rank": 35, "gower_w": importances[w_type], "w_type": w_type}
        for w_type in w_types
    ],
    "EigenRec": [{"gower": True, "rank": 50}, {"gower": False, "rank": 50}]
    + [
        {"gower": True, "rank": 50, "gower_w": importances[w_type], "w_type": w_type}
        for w_type in w_types
    ],
}

In [19]:
result = []

for name, (build_fn, scoring_fn) in models.items():
    result += compare_models(
        configs=model_configs[name],
        build_fn=build_fn,
        scoring_fn=scoring_fn,
        training=training,
        testset=testset,
        holdout=holdout,
        data_description=data_description,
        model_name=name,
    )

pd.DataFrame(result)

Running EigenRec: 100%|██████████| 5/5 [00:13<00:00,  2.77s/it]


,model,is_gower,w_type,HR,MRR,Coverage
0,knn,True,None,0.003926,0.001064,0.105968
1,knn,False,None,0.006169,0.002085,0.270496
2,knn,True,builtin,0.005609,0.001493,0.178472
3,knn,True,shap,0.007291,0.001584,0.146960
4,knn,True,permutation,0.004487,0.001287,0.161461
5,EASER,True,None,0.064498,0.023526,0.288622
6,EASER,False,None,0.064498,0.023526,0.288622
7,EASER,True,builtin,0.064498,0.023621,0.288622
8,EASER,True,shap,0.064498,0.023527,0.288901
9,EASER,True,permutation,0.064498,0.023527,0.288622


In [20]:
model_evaluate(
    topn_recommendations(random_scoring(testset, data_description), 10),
    holdout,
    data_description,
    10,
)

EvaluationMetrics(HR=0.0005608524957936063, MRR=0.0005608524957936063, Coverage=0.9949804796430564)